### Packages

In [22]:
# --- Packages ---
#
import XLSX
import DataFrames
import Dates

using JuMP, Gurobi
using ArgParse, Random

include("parameters.jl")
include("post_process.jl")

add_var_to_XLSX (generic function with 1 method)

In [23]:
# --- Run time-dependent formulation ---
#
include("DNEP_time_formulation.jl")

sets  = (N, Ns, Ns_init, K, L, L_init, Y, T, Omega_sending, Omega_receiving)
costs = (sub_expan_cost, sub_install_cost, line_cost, losses_cost, DSO_INTEREST_RATE)
substation_param = (S_rating_init, S_rating_max)
conductor_param  = (max_current, conductance, susceptance)
demand           = (load_profiles, tan_phi)
     
var_values, var_sets = time_dependent_formulation(sets, costs, substation_param, conductor_param, demand, delta_t)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-10
Set parameter TimeLimit to value 100
Set parameter Presolve to value 0
Set parameter TimeLimit to value 100
Set parameter Presolve to value 0
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[arm])

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 4854 rows, 3662 columns and 14129 nonzeros
Model fingerprint: 0xae691d1d
Model has 264 quadratic constraints
Variable types: 3650 continuous, 12 integer (12 binary)
Coefficient statistics:
  Matrix range     [2e-04, 4e+07]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [1e+03, 7e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-05, 2e+00]
Variable types: 3650 continuous, 12 integer (12 binary)

Root relaxation: objective 2.000000e+03, 1872 iterations, 0.02 seconds (0.02 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work

(Dict{String, Array{Float64}}("Q_G" => [2.7758118908668367e-5 0.0 0.0 0.0; 5.994291661915802e-5 0.0 0.0 0.0; … ; 0.0004305148193431839 0.0 0.0 0.0; 9.496038264549205e-5 0.0 0.0 0.0], "P_G" => [0.0 0.00013670000000000002 0.0 0.0; 0.00029519999999999997 0.0 0.0 0.0; … ; 0.0021201500000000003 0.0 0.0 0.0; 0.00046765 0.0 0.0 0.0], "P_cond_forward" => [1.0676200629674028e-5 0.00013670001538912402 -0.0012159288003772417; 0.00027214503443114257 -6.821210263296962e-13 0.0013506714851700963; … ; 0.0021201500269967255 0.0 0.0028681954834155476; 0.00046765 -6.821210263296962e-13 0.001501945484733369;;; 0.0 -1.5389124002895826e-11 0.0; -5.4569682106375694e-12 -3.637978807091713e-12 7.475731308659306e-5; … ; -2.699672473419068e-11 0.0 0.0; 0.0 0.0 0.0;;; -1.0676200629674025e-5 0.0 0.0; 2.305497102582561e-5 4.320099833421409e-12 0.0; … ; 0.0 0.0 -1.6370904631912708e-11; 0.0 0.0 -1.7280399333685637e-11], "P_losses" => [0.0, 0.0, -7.275957614183426e-12, 0.0, 0.0, 0.0, -7.275957614183426e-12, 0.0, 0.0,

In [14]:
println(size(var_values["P_cond_forward"]))

(24, 3, 3)


In [24]:
# --- Get all variables from time-depent model ---
#
var = var_values["X_i_ij"]


start_date = DateTime(today() + Day(1))
end_date   = DateTime(start_date + Day(1))
XLSX_PATH  = "output.xlsx"
# period covered
date_range = start_date:Minute(GRANULARITY*period):(end_date-Second(1))
for (key, value) in var_values
    if ndims(value) <= 2
        add_var_to_XLSX(XLSX_PATH, value, key, var_sets[key], date_range)

    elseif ndims(value) == 3
        processed_var = process3D_variable(value)
        if key == "P_cond_backward"
            println(size(processed_var))
        end
        add_var_to_XLSX(XLSX_PATH, processed_var, key, var_sets[key], date_range)

    elseif ndims(value) == 4
        X_i_ij = process_X_i_ij(value, Omega_receiving, Omega_sending)
        add_var_to_XLSX(XLSX_PATH, X_i_ij, "X_i_ij", var_sets[key], date_range)
    end
end

(24, 3)
24×5 DataFrame
 Row │ date        time      line_1       line_2        line_3       
     │ Any         Any       Any          Any           Any          
─────┼───────────────────────────────────────────────────────────────
   1 │ 04-04-2023  00:00:00  0.00129453   -0.0001367    -7.86e-5
   2 │ 04-04-2023  01:00:00  -0.00158973  -9.09495e-13  -0.0001309
   3 │ 04-04-2023  02:00:00  -0.00129453  -0.00032165   -0.00014825
   4 │ 04-04-2023  03:00:00  0.00109595   0.0           -8.285e-5
   5 │ 04-04-2023  04:00:00  0.000978229  -9.09495e-13  -0.0001429
   6 │ 04-04-2023  05:00:00  -0.00148528  5.68434e-14   -0.00014175
   7 │ 04-04-2023  06:00:00  -0.00129453  -0.000354617  -0.00021405
   8 │ 04-04-2023  07:00:00  0.000436029  0.0           -0.000449267
   9 │ 04-04-2023  08:00:00  0.000454179  -9.09495e-13  -0.0002575
  10 │ 04-04-2023  09:00:00  -0.00303798  5.68434e-14   -0.0013445
  11 │ 04-04-2023  10:00:00  -0.00129453  -0.000428067  -9.3e-5
  12 │ 04-04-2023  11:00:00  0.

In [5]:
# user enters the simulation data
start_date = DateTime(today() + Day(1))
end_date   = DateTime(start_date + Day(1))
println(DateTime(today()))
# period covered
date_range = start_date:Minute(GRANULARITY*period):(end_date-Second(1))
for (key, value) in var_values
    if ndims(value) <= 2
        add_var_to_XLSX("output.xlsx", value, key, var_sets[key], date_range)

    elseif ndims(value) == 3
        processed_var = process3D_variable(value)
        add_var_to_XLSX("output.xlsx", processed_var, key, var_sets[key], date_range)

    elseif ndims(value) == 4
        # note sure this has many sense
        X_i_ij, X_j_ij = process_X_i_ij(value)
        add_var_to_XLSX("output.xlsx", X_i_ij, "X_i_ij", var_sets[key], date_range)
        add_var_to_XLSX("output.xlsx", X_j_ij, "X_j_ij", var_sets[key], date_range)

    end


end

2023-03-29T00:00:00


In [ ]:

function process_3Dvariable(var::Array{Float64, 3})
    # 1st dimension: T, 2nd dimension: L, 3rd dimension: K
    T, L, ~ = size(var)
    processed_var = Array{Float64}(undef, T, L)
    # Trick to remove the kth dimension
    for t in 1:T
        processed_var[t, :] = sum(var[t, :, :], dims=2) 
    end    

    return processed_var
end

function process_X_i_ij(var::Array{Float64, 4})
    # 1st dim: T, 2nd dim: L, 3rd dim: K, 4th dim: N
    T, L, K, N = size(var)
    processed_var = Array{Float64}(undef, T, L, N)

    for t in T, i in N   
        processed_var[t,:,i] = sum(var[t, :, :, i],  dims=2)
    end

    X_i_ij = Array{Float64}(undef, T, L)
    X_j_ij = Array{Float64}(undef, T, L)

    for t in T, l in L
        ifrom = line_ends[l][1]
        ito = line_ends[l][2]
        X_i_ij[t,l] = processed_var[t, l, ifrom]
        X_j_ij[t,l] = processed_var[t, l, ito]
    end

    return X_i_ij, X_j_ij
end

In [ ]:
X_i_ij, X_j_ij = process_X_i_ij(var_dict["X_i_ij"])
processed_var = process_3Dvariable(var_dict["P_cond_forward"])


In [ ]:
# --- Function that adds a variable to an XLSX file ---
# 2 assumtions for this function to work:
# var must be either a scalar, or a Vector type or an Array of dimension 2
function add_var_to_XLSX(XLSX_PATH::String, var, var_name::String, dimensions::Vector{String}, date_range::StepRange{DateTime, Minute})
    # First we need to check if the XLSX already exists
    nb_dim = ndims(var)
    isfile(XLSX_PATH) ? mode = "rw" : mode = "w"

    XLSX.openxlsx(XLSX_PATH, mode = mode) do xf
        # Mode = "w" only when the file must be created
        mode == "w" && XLSX.rename!(xf[1], var_name)
        mode == "rw" && !XLSX.hassheet(xf, var_name) && XLSX.addsheet!(xf, var_name)

        sheet = xf[var_name]

        if nb_dim  == 0
            column_names = [var_name]
            data = reshape([var], (1,:))
            
        elseif nb_dim == 1 || nb_dim == 2
            # this only works if we make the assumption that the type can be vextor, matrix or array 
            nb_dim == 1 ? last_col = var_name : last_col=[dimensions[2] * "_$i" for i in 1:size(var)[2]]
            if dimensions[1] == "time"
                dates = Dates.format.(Date.(date_range), "dd-mm-yyyy")
                times = Dates.format.(Time.(date_range), "HH:MM:SS")
                column_names = ["date"; "time"; last_col]
                data = hcat(dates, times, var)
            else
                column_names = [dimensions[1]; last_col] 
                data = hcat(1:size(var)[1], var)
            end
        end
        df=DataFrame(data, column_names)
        XLSX.writetable!(sheet, df)
    end
end

In [ ]:
# --- Testing the function : add_var_to_XLSX---
# Idea: add the units
var = [1, 3, 4, 5]
var_name = "b"
dimensions = ["line"]

# user enters the simulation data
start_date = DateTime(today() + Day(1))
end_date   = DateTime(start_date + Day(1))
println(DateTime(today()))
# period covered
data_range = start_date:Minute(GRANULARITY*period):(end_date-Second(1))
println(typeof(data_range))
add_var_to_XLSX("output.xlsx", var, var_name, dimensions, data_range)

### Parameters

In [ ]:
# ------ RANDOM TABLES --------
rng  = 1:10

X_ij_re = rand(rng, (nb_daily_steps, L_size, K_size))
X_ij_im = rand(rng, (nb_daily_steps, L_size, K_size))
X_i_ij  = rand(rng, (nb_daily_steps, L_size, K_size, N_size))
 
alpha = Matrix{Int64}(undef, L_size, K_size)

Random.seed!(3)
for l in L
    count_1 = 0
    for k in K
        if rand([0 1]) == 1 && count_1 == 0
            count_1 += 1
            alpha[l, k] = 1
        else
            alpha[l, k] = 0
        end
    end
end

println(alpha)

for t in T
    X_ij_re[t,:,:] .*= alpha
end

println(X_ij_re[1,:,:])
X_ij_re_flat = Array{Float64}(undef, nb_daily_steps, L_size)
# Trick to remove the kth dimension
for t in T
    X_ij_re_flat[t, :] = sum(X_ij_re[t,:,:], dims=2) 
end

println(X_ij_re_flat[1,:])

# Remove the kth dimension of the table 